In [ ]:
import os
from Bio import SeqIO, Seq
import pandas as pd

In [ ]:
# read the xlsx file and create DataFrame
uni_file = 'uniprot-filtered-organism__Bacillus+subtilis+(strain+168)+[224308]%2--.xlsx'
assert(os.path.exists(uni_file)) #  sanity check
df = pd.read_excel(uni_file)

In [ ]:
# print the DF head
df.head()
# df_prot['Transmembrane'][0]

In [ ]:
uni_null_count = df['Gene names  (primary )'].isna().sum()
df_prot = df[df['Gene names  (primary )'].notna()]

uni_prot = df_prot['Gene names  (primary )']

count_unique = uni_prot.nunique()
prot_names = list(uni_prot)
uni_prot.drop_duplicates(inplace= True)


print('Total genes in UniProtKB file: ',len(prot_names)+uni_null_count)
print('Number of named genes: ',len(prot_names))
print('Number of genes without name: ', uni_null_count)
print('Number of unique genes name: ',count_unique)
print('Number of Duplicates genes by name: ',len(prot_names)-count_unique)

In [ ]:
## read file and import details
gene_bank_file = 'BS168.gb'
 # making sure that the path is valid
assert(os.path.exists(gene_bank_file))

with open(gene_bank_file, "r") as input_handle:
    gen = SeqIO.parse(input_handle, "genbank")
    record_gb = next(gen)

features = record_gb.features
print(type(features))

print(len(features))

In [ ]:
# find all CDS in GeneBank file
features_prot = []
gb_null_counter = 0

for feature in features:
    if feature.type == 'CDS':
        if feature.qualifiers.get('gene') is not None:
            features_prot.append(feature.qualifiers.get('gene')[0])
        else:
            gb_null_counter+=1

total_gb_prot = gb_null_counter+len(features_prot)
# print(features_prot[0])
print('Total Protien genes in GeneBank file: ',total_gb_prot)
print('Number of named protiens in GB: ',len(features_prot))
print('Number of unnamed protiens in GB: ',gb_null_counter)

In [ ]:
# compare lists and create 3 lists

# same protien in both
same_prot = set(features_prot).intersection(set(prot_names))
# protien exist only in genebank file 
gb_only = set(features_prot).difference(set(prot_names))
# protien exist only in Uniport file 
uni_only = set(prot_names).difference(set(features_prot))

uni_only = pd.Series(list(uni_only))
uni_same = list(same_prot)

print('Same protiens in both files: ', len(same_prot))
print('Protiens in GeneBank only: ', len(gb_only))
print('Protiens in UniProt only: ', len(uni_only))  

print('Number of Duplicates in Same protiens list: ',len(features_prot)-len(same_prot)-len(gb_only))

check where is the difference come from

In [ ]:
# check protien in GB  
gb_same = []
gb_diff = []
gb_count = 0
for feature in features:
    if feature.type == 'CDS':
        if feature.qualifiers.get('gene') is not None:
            gb_count += 1  
            if feature.qualifiers.get('gene')[0] in list(same_prot):
                gb_same.append(feature)
            else:
                gb_diff.append(feature)

# print(len(gb_same), len(gb_diff))


try to get datafram from the original DF so the name is in the gene name list "prot name" above.

In [ ]:
# Data frame of the differences, e.g. gene only in UniPort.
df_diff = df_prot[df_prot['Gene names  (primary )'].isin(list(uni_only))]

# df_diff.count()
df_diff = df_diff.drop_duplicates(subset = 'Gene names  (primary )')
df_diff.count()


In [ ]:
# plot some data:
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import numpy as np

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

def plot_gene_pie(_data, _labels,_titles,_exp,_angle,_width):

        fig = plt.figure(figsize=(15,10))
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        fig.subplots_adjust(wspace=0)

        # large pie chart parameters
        ax1.pie(_data[0], autopct=make_autopct(_data[0]), startangle=_angle,
                labels=_labels[0], explode=_exp, shadow=True)

        # small pie chart parameters
        ax2.pie(_data[1], autopct=make_autopct(_data[1]), startangle=_angle,textprops={'size': 'smaller'},
                labels=_labels[1], radius=0.5, shadow=True, explode=_exp)

        ax1.set_title(_titles[0])
        ax2.set_title(_titles[1])

        # use ConnectionPatch to draw lines between the two plots
        # get the wedge data
        theta1, theta2 = ax1.patches[0].theta1, ax1.patches[0].theta2
        center, r = ax1.patches[0].center, ax1.patches[0].r

        # draw top connecting line
        x = r * np.cos(np.pi / 180 * theta2) + center[0]
        y = np.sin(np.pi / 180 * theta2) + center[1]
        con = ConnectionPatch(xyA=(- _width / 2, .5), xyB=(x, y),
                        coordsA="data", coordsB="data", axesA=ax2, axesB=ax1)
        con.set_color([0, 0, 0])
        con.set_linewidth(2)
        ax2.add_artist(con)

        # draw bottom connecting line
        x = r * np.cos(np.pi / 180 * theta1) + center[0]
        y = np.sin(np.pi / 180 * theta1) + center[1]
        con = ConnectionPatch(xyA=(- _width / 2, -.5), xyB=(x, y), coordsA="data",
                        coordsB="data", axesA=ax2, axesB=ax1)
        con.set_color([0, 0, 0])
        ax2.add_artist(con)
        con.set_linewidth(2)

        plt.show()

#show UniProtKB file data:
uni_file_count = len(prot_names)
total_uni_file = uni_file_count + uni_null_count
uni_file_dup = uni_file_count-count_unique

uni_chart = [[uni_file_count, uni_null_count],[count_unique, uni_file_dup]]
labels = [['Named', 'Unnamed'],['Unique', 'Duplicates']]
titles= ['Total Genes in UniPort','Named Genes']
explode = [0.1, 0]
angle = -180 * (uni_file_count/total_uni_file)
width = .2

plot_gene_pie(uni_chart, labels, titles, explode, angle, width)

#show GeneBank file details

total_named_prot_gb = len(features_prot)
total_unprot_gb = len(features)- total_gb_prot
total_gb = total_gb_prot + total_unprot_gb
gb_chart = [[total_gb_prot, total_unprot_gb],[total_named_prot_gb, gb_null_counter]]
labels = [['Protiens', 'Other Types'],['Named', 'Unnamed']]
titles= ['Total Genes in GeneBank','Protiens Genes']
explode = [0.1, 0]
angle = -180 * (total_gb_prot/total_gb)
width = .2

plot_gene_pie(gb_chart, labels, titles, explode, angle, width)

compare_chart = [len(same_prot), len(gb_only), len(uni_only)]
_labels = ['Same Protiens', 'GB Unique Protiens', 'UniProtKB Unique Protiens']
plt.bar(_labels, compare_chart, color= ['r','g','b'], width=0.5)
plt.title('Files Comparison')
plt.tight_layout()
plt.show()


question B

In [155]:
trans_df = df[df['Transmembrane'].notna()]
# trans_df.head()
trans_s = trans_df['Transmembrane']
trans_s[0]
# trans_df['Sequence']

'TRANSMEM 6..26;  /note="Helical";  /evidence="ECO:0000255"; TRANSMEM 37..57;  /note="Helical";  /evidence="ECO:0000255"; TRANSMEM 68..88;  /note="Helical";  /evidence="ECO:0000255"'

In [153]:
import regex as re
transmem_list = list(trans_s)
# # len(transmem_list)
# trans_pos = []
for trans in transmem_list:
    r1 = 

1987